https://htmlpreview.github.io/?https://github.com/CogStack/MedCATtutorials/blob/main/notebooks/specialised/Preprocessing_SNOMED_CT.html

In [ ]:
import pickle
import re
from pathlib import Path

from medcat.cat import CAT
from tqdm.notebook import tqdm

from discharge_summaries.schemas.mimic import Record

In [ ]:
DATA_DIR = Path.cwd().parent / "data"
GT_DATA_PATH = DATA_DIR / "train.pkl"

MODEL_PATH = Path.cwd().parent / "models" / "umls_sm_pt2ch_533bab5115c6c2d6"

In [ ]:
with open(GT_DATA_PATH, "rb") as in_file:
    gt_dataset = [Record(**record) for record in pickle.load(in_file)]
len(gt_dataset)

In [ ]:
cat = CAT.load_model_pack(MODEL_PATH)
# type_ids_filter = [
#     "T020",
#     "T190",
#     "T049",
#     "T019",
#     "T047",
#     "T050",
#     "T033",
#     "T037",
#     "T048",
#     "T191",
#     "T046",
#     "T184",
# ] + ["T005", "T007"]
# cui_filters = {
#     cui
#     for type_ids in type_ids_filter
#     for cui in cat.cdb.addl_info["type_id2cuis"][type_ids]
# }
# cat.cdb.config.linking["filters"]["cuis"] = cui_filters

In [ ]:
num_hits = 0
num_examples = 0
misses = set()
for sample in tqdm(gt_dataset):
    for para in sample.discharge_summary.bhc_paragraphs:
        if not para.heading:
            continue
        clean_heading = para.heading
        heading_match = re.search(r"[a-zA-Z]", para.heading)
        clean_heading = (
            para.heading[heading_match.start() :] if heading_match else para.heading
        )
        # clean_heading = clean_heading.replace('/', ' ')
        doc = cat(clean_heading)
        # if not(any(word in clean_heading.lower() for word in {"fen", "communication", "access", "code"})):
        num_examples += 1
        if doc.ents:
            num_hits += 1
        else:
            misses.add(clean_heading.lower().strip())

num_hits / num_examples

In [ ]:
misses